## Homework 5
This notebook contains the solutions for the fifth homework assignment of the Machine Learning Zoomcamp 2025 course, which can be viewed on github with the following [link](https://github.com/DataTalksClub/machine-learning-zoomcamp/blob/master/cohorts/2025/05-evaluation/homework.md). The final solutions notebook is submitted for grading at the following [link](https://courses.datatalks.club/ml-zoomcamp-2025/homework/hw05).
### Setup

In [18]:
import re

In [24]:
BOLD = "\033[1m"

### Q1. `uv` version
1. Install uv
2. Use --version to find which version of `uv` was installed

What's the version of uv?

In [11]:
uv_version_cl = !uv --version
uv_version = uv_version_cl[0].split()[1]
print(f"The `uv` version = {BOLD}{uv_version}")

The `uv` version = 0.9.5


### Q2. Use `uv` to install Scikit-Learn
Use `uv` to install Scikit-Learn version 1.6.1.

What's the first hash for Scikit-Learn you get in the lock file?
* Include the entire string starting with sha256
* Don't include quotes

In [1]:
!uv init

Initialized project `05-deployment`


In [2]:
!uv add "scikit-learn==1.6.1"

Using CPython 3.12.1 interpreter at: /home/codespace/.python/current/bin/python3.12
Creating virtual environment at: .venv
Resolved 6 packages in 242ms                                         
Prepared 5 packages in 1.04s                                             
░░░░░░░░░░░░░░░░░░░░ [0/5] Installing wheels...                                 warning: Failed to hardlink files; falling back to full copy. This may lead to degraded performance.
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 5 packages in 386ms                               
 + joblib==1.5.2
 + numpy==2.3.4
 + scikit-learn==1.6.1
 + scipy==1.16.2
 + threadpoolctl==3.6.0


In [25]:
fname = "uv.lock"
is_package = False
is_skl = False
with open(fname, 'r') as f:
    for line in f:
        if line.startswith("[[package]]"):
            is_package = True
        if line.startswith('name = "scikit-learn"'):
            is_skl = True
        if is_package and is_skl and "hash" in line:
            hash_line = line
            break

In [26]:
match = re.search(r"hash = (.*?)(?:,|$)", hash_line)
hash_str = match.group(1).strip().replace('"', '')
print(f"The first hash value for the scikit-learn 1.6.1 package = {BOLD}{hash_str}")

The first hash value for the scikit-learn 1.6.1 package = sha256:b4fc2525eca2c69a59260f583c56a7557c6ccdf8deafdba6e060f94c1c59738e


### 3. Load model and score record
1. Write a script for loading the pipeline with pickle
2. Score this record: {
    "lead_source": "paid_ads",
    "number_of_courses_viewed": 2,
    "annual_income": 79276.0
}

What's the probability that this lead will convert?